 Les fonctions basiques :

In [2]:
from copy import deepcopy
import time

etat_depart = [[1, 2, 3], [8, 6, 0], [7, 5, 4]]
etat_but = [[1, 2, 3], [8, 0, 4], [7, 6, 5]]

# bas, droite, haut, gauche
DEPLACEMENTS = [(1, 0), (0, 1), (-1, 0), (0, -1)]

def est_etat_final(etat, but):
    return etat == but

def position_case_vide(etat):
    """Retourne la position (ligne, colonne)"""
    for row in range(len(etat)):
        for col in range(len(etat[row])):
            if etat[row][col] == 0:
                return row, col

def permuter_cases(etat, pos_vide, nouvelle_pos):
    etat_nouvel = deepcopy(etat)
    i, j = pos_vide
    x, y = nouvelle_pos
    etat_nouvel[i][j], etat_nouvel[x][y] = etat_nouvel[x][y], etat_nouvel[i][j]
    return etat_nouvel

def afficher_taquin(etat):
    print()
    for row in etat:
        print("+-----" * len(row) + "+")
        for cell in row:
            if cell != 0:
               print('| ',cell,' ',end="")
            else:
               print('|     ', end="")
        print("|")
    print("+-----" * len(etat[0]) + "+")

def valide_coordonnees(x,y):
    return 0 <= x < 3 and 0 <= y < 3

def transitions(etat): #transition de la case vide et retourne le nouveau taquin
    """Retourne les états possibles en déplaçant la case vide
  Cette fonction retourne une liste contenant les permutations possible de l'état donné de la liste etatDepart"""
    pos_vide = position_case_vide(etat)
    liste_permutations = []
    for dx, dy in DEPLACEMENTS:
        x, y = pos_vide[0] + dx, pos_vide[1] + dy
        if valide_coordonnees(x, y):
            liste_permutations.append(permuter_cases(etat, pos_vide, (x, y)))
    return liste_permutations

def filtrer_etats(liste, reference, inverse=False):
    """
    Supprime de la liste les éléments qui sont (ou non) dans la référence.
    Si `inverse` est True, supprime les éléments qui ne sont pas dans la référence.
    """
    return [etat for etat in liste if (etat in reference) == inverse]

def backtracking(closed_nodes):
    """Reconstitue le chemin vers l'état initial à partir des états explorés."""
    print("Début de la recherche du chemin vers la solution...\n")
    start_time = time.time()
    route = []
    etat_actuel = deepcopy(etat_but)

    while True:
        route.append(etat_actuel)
        possibles = transitions(etat_actuel)
        possibles = filtrer_etats(possibles, closed_nodes, inverse=True)

        if etat_depart in possibles:
            route.append(etat_depart)
            break

        etat_actuel = min(possibles, key=lambda x: closed_nodes.index(x))

    print(f"Solution trouvée en {len(route)} étapes.\n")
    for i, etat in enumerate(reversed(route), 1):
        print(f"\nÉtape : {i}")
        afficher_taquin(etat)
    print(f"Temps de recherche : {time.time() - start_time:.4f} secondes.\n")

_______________________________________________________________________________________________________________________________________________________________________

Recherche A* (A-Star Search)

In [6]:
import heapq
import time
from copy import deepcopy

def h(t,tf):
  """fonction heuristique pour calculer le nombre des cases mal placés sans tenir compte du case vide."""
  cases_malplacees = 0
  for i in range(len(t)):
    for j in range(len(t[i])):
      if (t[i][j] != tf[i][j])and (t[i][j] != 0):
        cases_malplacees=cases_malplacees+1
  return cases_malplacees

def aetoile(etat_depart,etat_but):
  start_time = time.time() 
  priority_queue = []
  heapq.heappush(priority_queue, (0, etat_depart, 0)) # (f(x), state, g(x))
  visited = set() 
  parent_dict = {} #records which state led to another during the search process so we can reconstruct the path from the initial state to the goal state after the search completes

  while priority_queue:
    f, current_state, current_level = heapq.heappop(priority_queue) #etat avec la plus petite valeur de f(x)
    print ('\n** pas', current_level,': **')
    afficher_taquin(current_state)

    if(current_state == etat_but):
      print("\n --> Goal trouvé après ", current_level, " itérations")
      print(f"Temps d'exécution : {time.time() - start_time:.3f} secondes.")
      return
    
    visited.add(tuple(map(tuple, current_state))) #convert list into a tuple of tuples so it can be stored in a set

    transitions_possibles = transitions(current_state)
    for next_state in transitions_possibles:
      if tuple(map(tuple, next_state)) in visited:
        continue

      #calcule g(x) et h(x)
      g_val = current_level + 1
      h_val = h(next_state, etat_but)
      f_val = g_val + h_val

      heapq.heappush(priority_queue, (f_val, next_state, g_val))
      parent_dict[tuple(map(tuple, next_state))] = current_state # converting state (2D list: mutable) to tuple of tuples (immutable structure) so it can be used as a key

  print("Recherche non conclussive. Aucun chemin trouvé.")
  print(f"Temps écoulé : {time.time() - start_time:.5f} secondes.")

print (h(etat_depart,etat_but)," cases mal placées depuis l'état initial.")
aetoile(etat_depart,etat_but)

3  cases mal placées depuis l'état initial.

** pas 0 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 1 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |  5  |     |
+-----+-----+-----+

** pas 2 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |     |  5  |
+-----+-----+-----+

** pas 3 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |     |  4  |
+-----+-----+-----+
|  7  |  6  |  5  |
+-----+-----+-----+

 --> Goal trouvé après  3  itérations
Temps d'exécution : 0.001 secondes.


_______________________________________________________________________________________________________________________________________________________________________
Recherche en profondeur (Depth-First Search - DFS)

In [8]:
trace= []  #les états parcourus dans l'ordre de l’état initial à l’état final pour reconstituer et afficher le chemin parcouru pendant la recherche
visited= []  #les noeuds déjà explorés
success= False 

def dfs(t, tf):
  global success 

  if success:
    return #pour eviter de continuer l'exploration d'autres branches si l'etat final a deja été trouvé dans la branche precedente

  if (t not in visited): #avant d'explorer nouvel état (t), on vérifie s'il est déjà dans visited
    trace.append(t)
    visited.append(t) 

    if est_etat_final(t,tf):
      success=True
      return
    
    for voisin in transitions(t):
      if voisin not in visited and not success:
         dfs(voisin,tf)

start = time.time()       
dfs(etat_depart,etat_but) 

for i, etat in enumerate(trace):
    print(f"\n** Pas {i} : **")
    afficher_taquin(etat)

print("\n --> Goal trouvé aprés",len(trace) - 1," iterations .")
print ('Time spent: %0.4fs' % (time.time()-start))



** Pas 0 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** Pas 1 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |  5  |     |
+-----+-----+-----+

** Pas 2 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |     |  5  |
+-----+-----+-----+

** Pas 3 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |     |  4  |
+-----+-----+-----+
|  7  |  6  |  5  |
+-----+-----+-----+

 --> Goal trouvé aprés 3  iterations .
Time spent: 0.0010s


_______________________________________________________________________________________________________________________________________________________________________
Recherche en largeur d'abord : (Breadth-First Search - BFS)

In [11]:
def bfs(t, tf):
  global start
  start=time.time()
  visited=[t]  
  queue=[t] #les états à parcourir 

  trace=[] #les états parcourus partant de l’état initial à l’état final 

  while queue: #queue n’est pas vide
    current_state=queue.pop(0) #current_state reçoit le 1er élément de queue et l’élimine de cette liste
       
    if est_etat_final(current_state,tf):
      trace.append(current_state)
      break 
    
    trace.append(current_state)

    for transition_possible in transitions(current_state):
      if transition_possible not in visited:
        visited.append(transition_possible)
        queue.append(transition_possible)

  return trace

niveaux=0
trace=bfs(etat_depart,etat_but)
for i in trace : 
   print()
   print ('** pas', niveaux,': **')
   afficher_taquin(i)
   niveaux+=1
print("\n --> Goal trouvé aprés",niveaux-1," iterations . \n")
print ('Time spent: %0.3fs' % (time.time()-start))


** pas 0 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 1 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |  5  |     |
+-----+-----+-----+

** pas 2 : **

+-----+-----+-----+
|  1  |  2  |     |
+-----+-----+-----+
|  8  |  6  |  3  |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 3 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |     |  6  |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 4 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |     |  5  |
+-----+-----+-----+

** pas 5 : **

+-----+-----+-----+
|  1  |     |  2  |
+-----+-----+-----+
|  8  |  6  |  3  |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 6 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-

_______________________________________________________________________________________________________________________________________________________________________
Recherche en profondeur limitée (Depth-Limited Search - DLS)

In [11]:
def dls(t, est_DFS, A_star, profondeur):
    global end
    start_time = time.time()
    freeNodes = [t]  #nœuds à explorer
    visited = []  

    # Si limit = -1, il n'y a pas de limite
    if profondeur != -1 :
        profondeur_actuelle = [0] 
    else :
        profondeur_actuelle = []
    
    print("L'état Initial")
    afficher_taquin(freeNodes[0])

    success = False
    iterations = 0
    
    while freeNodes:
        iterations += 1
        current = freeNodes.pop(0)  
        visited.append(current)

        # Vérification de la profondeur limite
        if profondeur != -1 and profondeur_actuelle[0] >= profondeur:
            if est_etat_final(current, etat_but):
                success = True
                print("\nEtat Final Atteint !")
                afficher_taquin(current)
                visited.append(current)
            if success:
                break
            continue

        possible_transitions = transitions(current)
        for next_state in possible_transitions:
            if est_etat_final(next_state, etat_but):
                success = True
                print("\nEtat Final Atteint !")
                afficher_taquin(next_state)
                visited.append(next_state)
                break

        if success:
            break

        # Mise à jour des nœuds à explorer
        if est_DFS:
            freeNodes = possible_transitions + freeNodes  # DFS : on explore d'abord les transitions
            if profondeur != -1:
                profondeur_actuelle = [profondeur_actuelle[0] + 1] * len(possible_transitions) + profondeur_actuelle  # Mise à jour du niveau
        else:
            freeNodes += possible_transitions  # BFS : on explore en élargissant le niveau
            if profondeur != -1:
                profondeur_actuelle += [profondeur_actuelle[0] + 1] * len(possible_transitions)

        # pour A*, les nœuds sont triés selon une  fonction heuristique
        if A_star:
            freeNodes.sort(key=lambda e: (iterations + h(e)))

    if success:
        print(f"\nLa recherche a pris {time.time() - start_time:.4f} secondes.\n")
        backtracking(visited)
    else:
        print("Aucune solution trouvée, limite atteinte : ", profondeur)
        
limit = 20000
dls(etat_depart, True, False, limit)

L'état Initial

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+
Aucune solution trouvée, limite atteinte :  20000
